In [2]:
from turkish_morphology import decompose, analyze

word = "kitaplarımızdaki"
word = "yapamadıklarımızdan"
sentence = "Yazarın kitaplarını yayın hakkını elinde bulunduran Aziz Nesin Vakfı'ndan yapılan açıklamaya göre,ilki 1946 yılında basılan Aziz Nesin'in kitapları, o dönemden bu yana 10 milyon 780 bin adet basıldı ve dağıtıldı"
word = "açıklayamamak"

def get_morphemes(word, analysis_index=-1):
    analyses = analyze.surface_form(word, use_proper_feature=False)
    
    if analysis_index >= len(analyses):
        analysis_index = -1

    print(f"Number of analyses: {len(analyses)}")

    if not analyses:
        return []
    
    print(analyses)

    decomposition = decompose.human_readable_analysis(analyses[analysis_index])

    morphemes = []

    for ig in decomposition.ig:
        root = getattr(ig, "root")
        if root:
            morpheme = root.morpheme
            if morpheme:
                morphemes.append(morpheme)

        if hasattr(ig, "derivation"):
            morpheme = getattr(ig.derivation, "meta_morpheme")
            if morpheme:
                morphemes.append(morpheme)

        if hasattr(ig, "inflection"):
            for inflection in ig.inflection:
                morpheme = getattr(inflection, "meta_morpheme")
                if morpheme:
                    morphemes.append(morpheme)

    return morphemes

def get_morphemes_from_sentence(sentence, analysis_index=-1):
    sentence_morphemes = []
    for word in sentence.split():
        sentence_morphemes.append(get_morphemes(word, analysis_index=analysis_index))
    return sentence_morphemes

# get_morphemes_from_sentence(sentence, analysis_index=8)
get_morphemes(word, analysis_index=-1)

Number of analyses: 6
['(açık[NN]+[PersonNumber=A3sg]+[Possessive=Pnon]+[Case=Bare])([VB]-lA[Derivation=Make])([VB]-YA[Derivation=Able]+mA[Polarity=Neg])([NOMP]-mAk[Derivation=Inf]+[PersonNumber=A3sg]+[Possessive=Pnon]+[Case=Bare]+[Copula=PresCop]+[PersonNumber=V3pl])', '(açık[NN]+[PersonNumber=A3sg]+[Possessive=Pnon]+[Case=Bare])([VB]-lA[Derivation=Make])([VB]-YA[Derivation=Able]+mA[Polarity=Neg])([NOMP]-mAk[Derivation=Inf]+[PersonNumber=A3sg]+[Possessive=Pnon]+[Case=Bare]+[Copula=PresCop]+[PersonNumber=V3sg])', '(açık[NN]+[PersonNumber=A3sg]+[Possessive=Pnon]+[Case=Bare])([VB]-lA[Derivation=Make])([VB]-YA[Derivation=Able]+mA[Polarity=Neg])([NOMP]-mAk[Derivation=Inf]+[PersonNumber=A3sg]+[Possessive=Pnon]+[Case=Nom]+[Copula=PresCop]+[PersonNumber=V3pl])', '(açık[NN]+[PersonNumber=A3sg]+[Possessive=Pnon]+[Case=Bare])([VB]-lA[Derivation=Make])([VB]-YA[Derivation=Able]+mA[Polarity=Neg])([NOMP]-mAk[Derivation=Inf]+[PersonNumber=A3sg]+[Possessive=Pnon]+[Case=Nom]+[Copula=PresCop]+[PersonNum

['açık', 'lA', 'YA', 'mA', 'mAk']

In [20]:
import pandas as pd

data = pd.read_csv("data/bilkent-turkish-writings/bilkent-turkish-writings-dataset.csv")
data.head()

,Unnamed: 0,text
0,0,BATUHAN ERDURCAN \r\n21301855 \r\nTURK 101-13 ...
1,1,\r\nYalçın Arslan \r\n21300458 \r\nAslı Uçar ...
2,2,Sudenur SOYSAL \r\nGÜZELLİK Mİ ÇİRKİNLİK Mİ HA...
3,3,Mars’ta Yaşam mı? \r\n \r\nHayatım boyunca evr...
4,4,"Bir Şehrin İki Yakasında \r\n \r\n""İki büyük c..."


In [22]:
import re
from utils import write_json

essays = []

for i, row in data.iterrows():
    if isinstance(row["text"], str):
        raw_texts = re.split("[.?!]\s",row["text"].replace("\r\n", ""))
        texts = []

        for text in raw_texts:
            texts.extend(re.split("\s\s\s\s", text))

        texts = [re.sub("^[A-ZÜÖĞIŞÇİ0-9\-\s.]+$", "", text) for text in texts]
        texts = [re.sub("\s+", " ", text).strip() for text in texts if all(["http" not in text.lower(), "kaynakça" not in text.lower()]) and len(text.strip().split()) > 5]
        
        if len(texts) > 1:
            essays.append({
                "id": f"bilkent-turkish-writings-{i}",
                "sentences": texts
            })

write_json(essays, "data/bilkent-turkish-writings/bilkent-turkish-writings-dataset.json", ensure_ascii=False)
